In [15]:
!pip install tensorflow_hub
!pip install keras tf-models-official pydot graphviz

     |████████████████████████████████| 1.6MB 22.8MB/s 
     |████████████████████████████████| 174kB 32.6MB/s 
     |████████████████████████████████| 358kB 36.7MB/s 
     |████████████████████████████████| 686kB 30.1MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 38.2MB 80kB/s 
     |████████████████████████████████| 1.2MB 37.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=a478ed903a0c1c208fc825854aea03b56f39ce9021e7ce2340b5d2da5e60daab
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22258 sha256=7977f45ec483e2b323324287bd0add432e5530d45b3d6a8a0959c762cd18d72f
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134

In [16]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from keras.utils import np_utils

import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization as tokenization

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
new_model=tf.keras.models.load_model ('/content/drive/MyDrive/twitter_bert')

In [17]:
tokenizerSaved = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join('/content/drive/MyDrive/twitter_bert', 'assets/vocab.txt'),
    do_lower_case=False)

In [18]:

def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)
def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [55]:
tweet = ['SLEEPY JOE BIDEN IS PROPOSING THE BIGGEST TAX HIKE IN OUR COUNTRIES HISTORY! CAN ANYBODY REALLY VOTE FOR THIS?']
tweets=[' Safeway is very rock n roll tonight ']
inputs = bert_encode(string_list=list(tweets), 
                     tokenizer=tokenizerSaved, 
                     max_seq_length=240)

In [56]:
prediction= new_model.predict(inputs)
print(prediction)
print('Tweet is', 'positive' if np.argmax(prediction)>0.5 else 'negative')

[[0.01137035 0.9886297 ]]
Tweet is positive
